In [1]:
import numpy as np
import random
import cv2
from libsvm.svm import svm_problem
from libsvm.svmutil import svm_train, svm_predict

train_ratio = 0.7
random_seed = 1143583
np.random.seed(random_seed)

# Function to randomly select subjects
def random_subject(num, start, end):
    np.random.seed(114)
    idx = random.sample(range(start, end), num)
    return sorted(idx[:num])

# Function to get train and test lists
def train_test_list(input_list, ratio, seed=random_seed):
    input_len = len(input_list)
    train_idx = random_subject(num=round(input_len*ratio), start=0, end=input_len)
    #for i in range(input_len):
        #if i in train_idx:
            #train_list = input_list[i]
        #else:
            #test_list = input_list[i]
    train_list = [input_list[i] for i in train_idx]
    test_list = [input_list[i] for i in range(input_len) 
                 if i not in train_idx]

    return train_list, test_list

# Function to get lists for PIE dataset
def pie_set(data_idx):
    pie_list = []
    pie_train_list, pie_test_list = [], []

    for subj_idx in data_idx:
        subj_list = [f'PIE/{subj_idx}/{i+1}.jpg' for i in range(50)]
        subj_train_list, subj_test_list = train_test_list(subj_list, ratio=train_ratio, seed=random_seed)
        pie_train_list.extend(subj_train_list)
        pie_test_list.extend(subj_test_list)
        pie_list.extend(subj_list)
    return pie_list, pie_train_list, pie_test_list

# Function to get lists for self dataset
def self_set():
    self_list = [f'self/{i+1}.jpg' for i in range(10)]
    self_train_list, self_test_list = train_test_list(self_list, ratio=train_ratio, seed=random_seed)
    return self_list, self_train_list, self_test_list

# Function to load images and labels
def get_img_vector(input_list):
    img_v, labels = [], []

    for path in input_list:
        img = cv2.imread(path)

        if img is None:
            print(f"Error: Image at path {path} failed to load.")
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (32, 32))

        img_v.append(img)
        labels.append(int(path.split('/')[1]) if path.split('/')[0] == 'PIE' else 0)

    img_x = np.array(img_v).reshape(len(img_v), -1)
    labels_x = np.array(labels)
    
    return img_x, labels_x

# Randomly select subjects
data_idx = random_subject(num=25, start=1, end=68)
pie_list, pie_train_list, pie_test_list = pie_set(data_idx)
self_list, self_train_list, self_test_list = self_set()

print('Selected subjects:', data_idx)

# Function to perform PCA transformation
def PCA_transform(X, n_components):
    X = X.astype(np.float64)
    X_mean = np.mean(X, axis=0)
    X_centered = X - X_mean
    covariance_matrix = np.cov(X_centered, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    idx = np.argsort(-eigenvalues)
    eigenvectors = eigenvectors[:, idx][:, :n_components]
    
    X_pca = X_centered @ eigenvectors
    return X_pca, X_mean, eigenvectors

# Combine image paths for both datasets
list_img = pie_list + self_list
train_list = pie_train_list + self_train_list
test_list = pie_test_list + self_test_list

# Load and preprocess data
train_data, train_label = get_img_vector(train_list)
test_data, test_label = get_img_vector(test_list)

C = [0.01, 0.1, 1]  # penalty parameter
D = [80, 200]  # dimensionality

# Function to preprocess data and train SVM models
def SVM_KNN(data, label, test_data, test_label, C, dim):
    print('\nFor PCA data:')
    for c in C:
        print(f'Penalty parameter C = {c}')
        param = f'-s 0 -t 0 -c {c}'

        for d in dim:
            print(f'Dimensionality D is {d}')
            data_new, mean, eigenv = PCA_transform(data, n_components=d)
            test_data_new = (test_data - mean) @ eigenv
            prob = svm_problem(label, data_new.real.tolist())
            model = svm_train(prob, param)
            pred_label, pref_acc, pred_val = svm_predict(test_label, test_data_new.real.tolist(), model)

def SVM_raw_KNN(data, label, test_data, test_label, C):
    print('\nFor raw data:')
    for c in C:
        print(f'Penalty parameter C = {c}')
        param = f'-s 0 -t 0 -c {c}'
        prob = svm_problem(label, data)
        m = svm_train(prob, param)
        p_label, p_acc, p_val = svm_predict(test_label, test_data, m)                

SVM_raw_KNN(train_data, train_label, test_data, test_label, C)
SVM_KNN(train_data, train_label, test_data, test_label, C, dim=D)

D:\ANACONDA\Lib\site-packages\libsvm\svm.py:146: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


Selected subjects: [1, 3, 5, 6, 9, 12, 19, 21, 22, 31, 32, 36, 37, 43, 45, 47, 50, 51, 54, 58, 59, 62, 63, 65, 66]

For raw data:
Penalty parameter C = 0.01
Accuracy = 92.5926% (350/378) (classification)
Penalty parameter C = 0.1
Accuracy = 92.5926% (350/378) (classification)
Penalty parameter C = 1
Accuracy = 92.5926% (350/378) (classification)

For PCA data:
Penalty parameter C = 0.01
Dimensionality D is 80
Accuracy = 91.7989% (347/378) (classification)
Dimensionality D is 200
Accuracy = 93.1217% (352/378) (classification)
Penalty parameter C = 0.1
Dimensionality D is 80
Accuracy = 91.7989% (347/378) (classification)
Dimensionality D is 200
Accuracy = 93.1217% (352/378) (classification)
Penalty parameter C = 1
Dimensionality D is 80
Accuracy = 91.7989% (347/378) (classification)
Dimensionality D is 200
Accuracy = 93.1217% (352/378) (classification)
